### API by Acu-Trade

#### Estimate by car details

##### Get avail makes
Pass in "Year"
GET `https://ico.autotrader.ca/Ico/GetMakes?year=2023`
Response
```json
{
  "Makes": [
    { "Make": "ACURA" },
    { "Make": "ALFA ROMEO" },
    { "Make": "AUDI" },
    { "Make": "BMW" },
    { "Make": "BUICK" },
    { "Make": "CADILLAC" },
    { "Make": "CHEVROLET" },
    { "Make": "CHRYSLER" },
    { "Make": "DODGE" },
    { "Make": "FORD" },
    { "Make": "GENESIS" },
    { "Make": "GMC" },
    { "Make": "HONDA" }, // pass into next...
    { "Make": "HYUNDAI" },
    { "Make": "INFINITI" },
    { "Make": "JAGUAR" },
    { "Make": "JEEP" },
    { "Make": "KIA" },
    { "Make": "LAND ROVER" },
    { "Make": "LEXUS" },
    { "Make": "LINCOLN" },
    { "Make": "MASERATI" },
    { "Make": "MAZDA" },
    { "Make": "MERCEDES-BENZ" },
    { "Make": "MINI" },
    { "Make": "MITSUBISHI" },
    { "Make": "NISSAN" },
    { "Make": "PORSCHE" },
    { "Make": "RAM" },
    { "Make": "ROLLS-ROYCE" },
    { "Make": "SUBARU" },
    { "Make": "TOYOTA" },
    { "Make": "VOLKSWAGEN" },
    { "Make": "VOLVO" },
    { "Make": "WAGONEER" }
  ]
}
```

##### Get avail models
Pass in "Year", "Make"
GET `https://ico.autotrader.ca/Ico/GetModels?year=2023&make=HONDA`
Response
```json
{
  "Models": [
    { "Model": "ACCORD HYBRID" }, // pass into next...
    { "Model": "ACCORD SEDAN" },
    { "Model": "CIVIC HATCHBACK" },
    { "Model": "CIVIC SEDAN" },
    { "Model": "CIVIC SI SEDAN" },
    { "Model": "CR-V" },
    { "Model": "CR-V HYBRID" },
    { "Model": "HR-V" },
    { "Model": "ODYSSEY" },
    { "Model": "PASSPORT" },
    { "Model": "PILOT" },
    { "Model": "RIDGELINE" }
  ]
}
```

##### Get avail trims
Pass in "Year", "Make", "Model"
GET `https://ico.autotrader.ca/Ico/GetTrims?year=2023&make=HONDA&model=ACCORD%20HYBRID`
Response
```json
{
  "Versions": [
    {
      "Trim": "SPORT ECVT",
      "Guid": "C438174", // pass into next...
      "Specialized": false,
      "Source": 1 // pass into next...
    },
    {
      "Trim": "TOURING ECVT",
      "Guid": "C448275",
      "Specialized": false,
      "Source": 1
    }
  ]
}
```

##### Validate trim
Pass in "Guid", "Source"
GET `https://ico.autotrader.ca/Ico/validateTrim?guid=C438174&source=1`
Response
```json
{ "IsValid": true }
```

##### Get estimated value from result page
Clear cookie before use
POST `https://www.autotrader.ca/valuations/ResultPage/6a1b1c2e-b1c3-468e-be7e-8afe393b4be9`
Header
```
Host: www.autotrader.ca
Content-Type: application/x-www-form-urlencoded
Content-Length: 45
```
Body `guid=C438174&mileage=20000&postalCode=L8B 1A1`
Use Regex `\$([0-9,]+) - \$([0-9,]+)` (1-min, 2-max) to find value range in response text.
